# Part B

## Step 1 - Adding Features

#### Load Data

In [ ]:
import pandas as pd

hotels_file_path = "./hotels_data.csv"  
df = pd.read_csv(hotels_file_path)

df.head()

#### Add columns and load to new CSV

In [ ]:
#Ensure dates are in correct format
df['Snapshot Date'] = pd.to_datetime(df['Snapshot Date'])
df['Checkin Date'] = pd.to_datetime(df['Checkin Date'])

df['DayDiff'] = (df['Checkin Date'] - df['Snapshot Date']).dt.days
df['WeekDay'] = df['Checkin Date'].dt.day_name()
df['DiscountDiff'] = df['Original Price'] - df['Discount Price']
df['DiscountPerc'] = (df['DiscountDiff'] / df['Original Price']) * 100

print(df.head())

#### Save To CSV

In [ ]:
changed_hotels_path = "./hotels_data_changed.csv"
df.to_csv(changed_hotels_path, index=False)

## Step 2 - Best Discount Code


**b. Classification Algorithms:** 
   - After creating this dataset, we will implement various classification algorithms in Python to predict the maximum discount code given the input parameters. 
   - Implement and evaluate the following five algorithms: Random Forest, Decision Tree, Naïve Bayes, XGBoost, and a simple Random classifier.
   - For each algorithm, you need to experiment with different parameter settings to find the optimal combination that yields the best performance. 
   - Explanation of how each chosen parameter affects the algorithm's performance. 

#### Data Preperation

In [ ]:
# Drop irrelevant columns

df = pd.read_csv(changed_hotels_path) 
df = df[['WeekDay', 'Snapshot Date', 'Checkin Date', 'DayDiff', 'Hotel Name', 'Discount Code']] 
df = df.rename(columns={'Discount Code': 'Class'})
df.head()

In [ ]:
# Feature Engineering

def map_days_to_numbers(df: pd.DataFrame) -> pd.DataFrame:
    weekday_mapping = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}
    df['WeekDay'] = df['WeekDay'].map(weekday_mapping)
    return df

def map_hotel_names_to_numbers(df: pd.DataFrame) -> pd.DataFrame:
    hotel_mapping = {hotel: i for i, hotel in enumerate(df['Hotel Name'].unique())}
    df['Hotel_Index'] = df['Hotel Name'].map(hotel_mapping)
    df.drop(['Hotel Name'], axis=1, inplace=True)
    return df

def map_date_to_numbers(df: pd.DataFrame, old_coloumn: str, new_coloumn: str) -> pd.DataFrame:
    df[old_coloumn] = pd.to_datetime(df[old_coloumn])
    df[f'{new_coloumn}_Year'] = df[old_coloumn].dt.year
    df[f'{new_coloumn}_Month'] = df[old_coloumn].dt.month
    df[f'{new_coloumn}_Day'] = df[old_coloumn].dt.day
    df.drop([old_coloumn], axis=1, inplace=True)

    return df

In [ ]:
df = map_days_to_numbers(df)
df = map_hotel_names_to_numbers(df)
df = map_date_to_numbers(df, 'Snapshot Date', 'Snapshot')
df = map_date_to_numbers(df, 'Checkin Date', 'Checkin')
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numerical_features = ['DayDiff', 'Snapshot_Year', 'Snapshot_Month', 'Snapshot_Day', 'Checkin_Year', 'Checkin_Month', 'Checkin_Day']
categorical_features = ['WeekDay', 'Hotel_Index']

# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 8))
axes = axes.flatten()

# Plot histograms for numerical features
for i, feature in enumerate(numerical_features):
  sns.histplot(data=df, x=feature, ax=axes[i])
  axes[i].set_title(f'Distribution of {feature}')

# Plot bar charts for categorical features (assuming enough data for each category)
for i, feature in enumerate(categorical_features):
  sns.countplot(data=df, x=feature, ax=axes[i])
  axes[i].set_title(f'Distribution of {feature}')

plt.tight_layout()
plt.show()

In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = df[['WeekDay', 'DayDiff', 'Hotel_Index', 'Snapshot_Year', 'Snapshot_Month', 'Snapshot_Day', 'Checkin_Year', 'Checkin_Month', 'Checkin_Day']]
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

#### Naive Bayes
We have the following feature types:

* **Continuous:**
    - `DayDiff` 
    - `Snapshot_Year`
    - `Snapshot_Month`
    - `Snapshot_Day`
    - `Checkin_Year`
    - `Checkin_Month`
    - `Checkin_Day`

* **Categorical:**
    - `WeekDay` 
    - `Hotel_Index`

**Naive Bayes Variants and Suitability:**

* **GaussianNB:**
    - **Best suited for continuous features.** It assumes that the features are normally distributed (Gaussian). Since we have several continuous features GaussianNB seems like the choice.

* **MultinomialNB:**
    - **Suitable for discrete features.** While `WeekDay` and `Hotel_Index` can be treated as categorical, they don't inherently represent counts or frequencies.

* **BernoulliNB:**
    - **Suitable for binary features.** Not applicable in this case as none of the features are binary.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
y_pred = naive_bayes.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred)) 

## Step 3 - Clustering Based on Price Polices 



Identify the 150 hotels with the most data in the dataset and extract their records.

In [ ]:
import pandas as pd

file_path = "./hotels_data.csv"  
df = pd.read_csv(file_path)

hotel_counts = df['Hotel Name'].value_counts()
top_150_hotels = hotel_counts.head(150).index
filtered_df = df[df['Hotel Name'].isin(top_150_hotels)]

filtered_df.head()

Find the 40 most common check-in dates  in the dataset and extract their records.


In [ ]:
checkin_counts = filtered_df['Checkin Date'].value_counts()
top_40_checkin_dates = checkin_counts.head(40).index
filtered_checkin_df = filtered_df[filtered_df['Checkin Date'].isin(top_40_checkin_dates)]

filtered_checkin_df.head()

## step 4 - Building a Predictive Model for Hotel Pricing Dynamics Using Snapshot Data



## step 5 - PySpark & Mllib for step 3